In [ ]:
from os import listdir, remove
from os.path import isfile, join
from PIL import Image
import numpy as np
import pandas as pd
import os
import cv2
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
def read_folders_color(path, start=0, end=100):
    elems = {}
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if not isfile(file_path):
            print(f'... {file_name} ...')
            elems[file_name] = read_paths_cv2_color(file_path, start, end)
    return elems

def read_paths_cv2_color(path, start=0, end=100):
    new_size = (128, 128)
    animals = []
    limit = start
    for file_name in listdir(path):
        file_path = join(path, file_name)
        if limit >= end:
            break
        if not isfile(file_path):
            print(f'WARN - ignoring {file_name}')
            continue

        try:
            img = img = cv2.imread(file_path)
            img_resized = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)
            if any('' in x for x in img_resized):
                print(f'ERROR - {file_path} - empty content')
            animals.append(img_resized)
        except:
            print(f'ERROR - {file_path}')
        limit += 1

    return animals


In [ ]:
# define a dict with animal - number
animal_names = {
    'butterfly': 0,
    'cat': 1,
    'chicken': 2,
    'cow': 3,
    'dog': 4,
    'elephant': 5,
    'horse': 6,
    'sheep': 7,
    'spider': 8,
    'squirrel': 9
}

In [ ]:
# create an array with all data to get the train - test
full_array = []
targets = []
for k,v in elems.items():
    full_array = full_array + v
    targets = targets + ([animal_names[k]] * len(v))

full_array = np.asarray(full_array)
targets = np.asarray(targets)

In [ ]:
##################
# OPTION 2
# rebase pixels from 0 - 255 to 0 - 1 vals

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(full_array, targets, test_size=0.2, random_state=73)

In [ ]:
# create model
# flatten shape
# test 1
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128)),
    keras.layers.Dense(16384, activation='relu'),
    keras.layers.Dense(4096, kernel_initializer='lecun_normal', activation='selu'),
    keras.layers.Dense(1024, kernel_initializer='lecun_normal', activation='selu'),
    keras.layers.Dense(256, kernel_initializer='lecun_normal', activation='selu'),
    keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'),
    keras.layers.Dense(10, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=4)